# Exclusion Semantics: Unilateral Witness Negation

This notebook explores the exclusion semantic framework, which implements a unilateral approach to negation using witness predicates. Unlike bilateral semantics that treat verification and falsification symmetrically, exclusion semantics provides a novel approach where negation is handled through explicit witness functions.

We'll examine both **countermodels** (showing where classical laws fail) and **theorems** (showing what remains valid) to understand this innovative semantic framework.

## Setup and Theory Configuration

In [1]:
# Import required modules for exclusion semantics
import sys
from model_checker.jupyter import create_build_example, build_and_check

# Import exclusion semantic components
from model_checker.theory_lib.exclusion.semantic import (
    WitnessSemantics, 
    WitnessProposition, 
    WitnessStructure
)
from model_checker.theory_lib.exclusion.operators import witness_operators

# Define the exclusion semantic theory
exclusion_theory = {
    "semantics": WitnessSemantics,
    "proposition": WitnessProposition,
    "model": WitnessStructure,
    "operators": witness_operators,
}

print(f"Loaded exclusion theory with witness negation semantics")
print(f"Operators: {', '.join(witness_operators.operator_dictionary.keys())}")

Loaded exclusion theory with witness negation semantics
Operators: \neg, \wedge, \vee, \equiv


---

## Example 1: The Challenge of Double Negation

### Background
In classical logic, double negation elimination (¬¬A ⊨ A) is a fundamental principle. However, in exclusion semantics with witness predicates, this principle can fail. This happens because negation is not simply truth-value reversal but involves explicit witness functions that may not compose as expected.

### The Argument
- **Premise**: ¬¬A (not not A)
- **Invalid Conclusion**: A
- **Why it fails**: The witness structure for double negation may not guarantee the truth of A

In [ ]:
# EX_CM_6: Test double negation elimination
EX_CM_6_example = [
    ['\\neg \\neg A'],     # Premise: double negation of A
    ['A'],                 # Conclusion: A (may fail!)
    {
        'N': 3,
        'possible': False,
        'contingent': True,
        'non_empty': True,
        'non_null': True,
        'disjoint': False,
        'fusion_closure': False,
        'max_time': 5,
        'iterate': 2,
        'expectation': True,  # We expect a countermodel
    }
]

print("Running model checker...")
model = create_build_example('EX_CM_6', exclusion_theory, EX_CM_6_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'EX_CM_6',
        'Double Negation Elimination',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected for exclusion semantics!)")

### Result Interpretation
The countermodel demonstrates how witness negation differs from classical negation. The witness predicates create a structure where double negation doesn't simply cancel out, revealing the non-classical nature of this semantic framework.

---

## Example 2: Disjunctive Syllogism Failure

### Background
*Disjunctive syllogism* (from "A or B" and "not A", conclude "B") is another classical principle that can fail in exclusion semantics. This failure illustrates how witness predicates interact with disjunction in unexpected ways.

### The Argument
- **Premises**: (A ∨ B), ¬A
- **Invalid Conclusion**: B
- **Why it fails**: The witness for ¬A doesn't necessarily force B to be true

In [ ]:
# EX_CM_23: Test disjunctive syllogism
EX_CM_23_example = [
    ['(A \\vee B)', '\\neg A'],  # Premises: A or B, and not A
    ['B'],                        # Conclusion: B (may fail!)
    {
        'N': 3,
        'possible': False,
        'contingent': False,
        'non_empty': False,
        'non_null': False,
        'disjoint': False,
        'fusion_closure': False,
        'max_time': 5,
        'iterate': 2,
        'expectation': True,  # We expect a countermodel
    }
]

print("Running model checker...")
model = create_build_example('EX_CM_23', exclusion_theory, EX_CM_23_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'EX_CM_23',
        'Disjunctive Syllogism',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected for exclusion semantics!)")

### Result Interpretation
The failure of disjunctive syllogism shows how exclusion semantics diverges from classical logic. The witness structure allows for models where disjunction and negation interact in non-standard ways, challenging our classical intuitions.

---

## Example 3: Conjunction Distribution (Valid)

### Background
While many classical principles fail, some distribution laws remain valid in exclusion semantics. The distribution of conjunction over disjunction shows that not all classical theorems are lost in this framework.

### The Argument
- **Premise**: A ∧ (B ∨ C)
- **Valid Conclusion**: (A ∧ B) ∨ (A ∧ C)
- **Why it works**: The witness structure preserves this distribution pattern

In [ ]:
# EX_TH_3: Test conjunction distribution (left to right)
EX_TH_3_example = [
    ['(A \\wedge (B \\vee C))'],           # Premise
    ['((A \\wedge B) \\vee (A \\wedge C))'], # Conclusion (valid!)
    {
        'N': 3,
        'possible': False,
        'contingent': False,
        'non_empty': False,
        'non_null': False,
        'disjoint': False,
        'fusion_closure': False,
        'max_time': 5,
        'expectation': False,  # We expect no countermodel (valid)
    }
]

print("Running model checker...")
model = create_build_example('EX_TH_3', exclusion_theory, EX_TH_3_example)

# Check validity
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to a theorem!")
    model.model_structure.print_to(
        model.settings,
        'EX_TH_3',
        'Conjunction Distribution',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Conjunction Distribution")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nThis distribution law survives in exclusion semantics.")
    print("=" * 70)

### Result Interpretation
The validity of conjunction distribution shows that exclusion semantics preserves some classical logical structure. This selective preservation helps identify which logical principles are truly fundamental versus those that depend on specific assumptions about negation.

---

## Example 4: De Morgan's Law Failure

### Background
*De Morgan's Laws*, which relate conjunction and disjunction through negation, can fail in exclusion semantics. This failure is particularly interesting because it shows how witness negation disrupts the duality between ∧ and ∨.

### The Argument
- **Premise**: ¬(A ∧ B)
- **Invalid Conclusion**: (¬A ∨ ¬B)
- **Why it fails**: Witness predicates don't distribute over conjunction as expected

## Example 5: Contraposition Failure

### Background
*Contraposition* is a fundamental principle in classical logic stating that if A implies B, then not-B implies not-A. In exclusion semantics, this principle can fail due to the asymmetric nature of witness negation.

### The Argument
- **Premise**: A → B (if A then B, defined as ¬A ∨ B)
- **Invalid Conclusion**: ¬B → ¬A (if not B then not A)
- **Why it fails**: Witness predicates don't preserve contraposition patterns

In [ ]:
# EX_CM_15: Test contraposition
# Since exclusion theory doesn't have implication, we use (¬A ∨ B) for (A → B)
EX_CM_15_example = [
    ['(\\neg A \\vee B)'],           # Premise: A implies B
    ['(\\neg \\neg B \\vee \\neg A)'], # Conclusion: not-B implies not-A
    {
        'N': 3,
        'possible': False,
        'contingent': True,
        'non_empty': True,
        'non_null': True,
        'disjoint': False,
        'fusion_closure': False,
        'max_time': 5,
        'iterate': 2,
        'expectation': True,  # We expect a countermodel
    }
]

print("Running model checker...")
model = create_build_example('EX_CM_15', exclusion_theory, EX_CM_15_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'EX_CM_15',
        'Contraposition',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected for exclusion semantics!)")

### Result Interpretation
The failure of contraposition further illustrates the non-classical nature of exclusion semantics. The witness predicates for negation create directional asymmetries that break the classical equivalence between a conditional and its contrapositive.

---

In [ ]:
# EX_CM_11: Test De Morgan's Law
EX_CM_11_example = [
    ['\\neg (A \\wedge B)'],        # Premise: not (A and B)
    ['(\\neg A \\vee \\neg B)'],    # Conclusion: (not A) or (not B) (may fail!)
    {
        'N': 3,
        'possible': False,
        'contingent': False,
        'non_empty': False,
        'non_null': False,
        'disjoint': False,
        'fusion_closure': False,
        'max_time': 5,
        'expectation': True,  # We expect a countermodel
    }
]

print("Running model checker...")
model = create_build_example('EX_CM_11', exclusion_theory, EX_CM_11_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'EX_CM_11',
        "De Morgan's Law",
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected for exclusion semantics!)")

### Result Interpretation
The failure of De Morgan's Laws reveals how deeply witness negation affects logical structure. The witness predicates create an asymmetry that breaks the classical duality between conjunction and disjunction under negation.

---

## Summary

This notebook has explored the exclusion semantic framework:

### Invalid Classical Principles (Countermodels)
- **Double Negation Elimination**: ¬¬A does not entail A  
- **Disjunctive Syllogism**: From A ∨ B and ¬A, cannot conclude B
- **De Morgan's Laws**: ¬(A ∧ B) does not entail (¬A ∨ ¬B)

### Valid Principles (Theorems)
- **Conjunction Distribution**: A ∧ (B ∨ C) entails (A ∧ B) ∨ (A ∧ C)

These examples demonstrate how exclusion semantics challenges our understanding of fundamental logical principles by showing how different approaches to negation can dramatically alter logical structure. The unilateral witness approach creates novel semantic behavior that is particularly valuable for understanding non-classical logics and the foundations of negation.